# How much time buses spend at stops

## Imports

In [1]:
import warnings
from datetime import datetime
from pathlib import Path

import geopandas as gpd
import gtfs_kit as gk
import pandas as pd
from google.transit.gtfs_realtime_pb2 import FeedMessage

## General settings

In [2]:
pb2_path = Path("../data")
sched_path = "../data/itm_south_east_gtfs.zip"

## Load pb2 data

In [3]:
trips = []
for f in sorted(pb2_path.glob("*.pb2")):
    msg = FeedMessage()
    msg.ParseFromString(f.read_bytes())
    for t in msg.entity:
        trips.append(t)

In [4]:
# 1 == STOPPED_AT
# 2 == IN_TRANSIT_TO
rows = [
    {
        "id": t.id,
        "trip_id": t.vehicle.trip.trip_id,
        "route_id": t.vehicle.trip.route_id,
        "start_time": t.vehicle.trip.start_time,
        "start_date": t.vehicle.trip.start_date,
        "latitude": t.vehicle.position.latitude,
        "longitude": t.vehicle.position.longitude,
        "current_stop": t.vehicle.current_stop_sequence,
        "current_status": t.vehicle.current_status,
        "timestamp": datetime.utcfromtimestamp(t.vehicle.timestamp),
        "vehicle": t.vehicle.vehicle.id,
    }
    for t in trips
]
df = pd.DataFrame(rows).drop_duplicates()
df.timestamp = df.timestamp.dt.tz_localize("UTC")

Convert to GeoDataFrame

In [5]:
df = (
    df.assign(geometry=gpd.points_from_xy(x=df.longitude, y=df.latitude))
    .drop(["longitude", "latitude"], axis=1)
    .pipe(gpd.GeoDataFrame, crs=4326)
)
df.head(2)

,id,trip_id,route_id,start_time,start_date,current_stop,current_status,timestamp,vehicle,geometry
0,13763630073698663407,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,50065,16:10:00,20221015,13,2,2022-10-15 16:37:46+00:00,300,POINT (-1.21231 51.72981)
1,15671302897479048672,VJa30a7b0edb53edbd19908a14241d6f9f8d43fd1a,50065,16:25:00,20221015,14,2,2022-10-15 16:37:43+00:00,311,POINT (-1.21449 51.73141)


In [6]:
tid = df.iloc[0].trip_id
trip = df.loc[df.trip_id == tid]

In [7]:
# trip.plot()

## Get bus stop location data

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fd = gk.read_feed(sched_path, dist_units="mi")

In [9]:
tid_trips = fd.trips.loc[fd.trips.trip_id == tid]
tid_trips

,route_id,service_id,trip_id,trip_headsign,block_id,shape_id,wheelchair_accessible,trip_direction_name,vehicle_journey_code
20611,50065,649,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,Minchery Farm,NaN,NaN,0,NaN,VJ_49-5A-O-y10-1-228-T2


In [10]:
tid_stop_times = fd.stop_times.loc[fd.stop_times.trip_id == tid]
tid_stop_times.head(6)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,stop_direction_name
648640,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,17:10:00,17:10:00,340001989S2,0,NaN,0,1,NaN,1,NaN
648641,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,17:15:00,17:15:00,340002074G5,1,NaN,0,0,NaN,1,NaN
648642,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,17:18:00,17:18:00,340001992K1,2,NaN,0,0,NaN,1,NaN
648643,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,17:19:00,17:19:00,340001126TYN,3,NaN,0,0,NaN,1,NaN
648644,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,17:20:00,17:20:00,340001199PEM,4,NaN,0,0,NaN,1,NaN
648645,VJ9319dc35a520096b9b1e6abe863fd465be3f5e80,17:21:00,17:21:00,340001198OUT,5,NaN,0,0,NaN,1,NaN


In [11]:
stops = (
    fd.stops.assign(
        geometry=gpd.points_from_xy(x=fd.stops.stop_lon, y=fd.stops.stop_lat)
    )
    .drop(["stop_lon", "stop_lat"], axis=1)
    .pipe(gpd.GeoDataFrame, crs=4326)
)
stops.geometry = stops.to_crs(epsg=3857).buffer(30).to_crs(epsg=4326)
stops = stops.loc[stops.stop_id.isin(tid_stop_times.stop_id)]
stops.head()

,stop_id,stop_code,stop_name,wheelchair_boarding,location_type,parent_station,platform_code,geometry
35761,340001992K1,oxfgjmtm,Queens Lane (Stop K1),0,NaN,NaN,NaN,"POLYGON ((-1.25126 51.75279, -1.25126 51.75277..."
35767,340001989S2,oxfgjmap,Speedwell Street (Stop S2),0,NaN,NaN,NaN,"POLYGON ((-1.25774 51.74848, -1.25774 51.74846..."
35793,340002074G5,oxfgwapd,St Aldates (Stop G5),0,NaN,NaN,NaN,"POLYGON ((-1.25719 51.75151, -1.25719 51.75149..."
35887,340001257BTW,oxfagwpd,The Original Swan,0,NaN,NaN,NaN,"POLYGON ((-1.21207 51.73377, -1.21207 51.73375..."
36218,340001126TYN,oxfadamp,The Plain,0,NaN,NaN,NaN,"POLYGON ((-1.24147 51.74920, -1.24147 51.74919..."


## And compare them!

In [12]:
df_map = pd.concat([trip[["geometry"]].assign(tp=0), stops[["geometry"]].assign(tp=1)])
# df_map.explore(tiles="CartoDB positron", cmap="viridis", column="tp")

In [13]:
cols = ["current_stop", "current_status", "timestamp", "stop_id", "stop_name"]
trip_stops = trip.sjoin(stops, how="left")[cols]
trip_stops = trip_stops.assign(cum_stop=trip_stops.groupby("stop_id").cumcount())
trip_stops.head()

,current_stop,current_status,timestamp,stop_id,stop_name,cum_stop
0,13,2,2022-10-15 16:37:46+00:00,NaN,NaN,NaN
24,14,2,2022-10-15 16:38:09+00:00,NaN,NaN,NaN
93,14,2,2022-10-15 16:38:46+00:00,NaN,NaN,NaN
162,15,2,2022-10-15 16:39:13+00:00,340001252OUT,Church Cowley School,0.0
231,15,2,2022-10-15 16:39:45+00:00,NaN,NaN,NaN


In [14]:
long_stops = trip_stops.loc[
    trip_stops.stop_id.isin(trip_stops.loc[trip_stops.cum_stop >= 1].stop_id.unique())
]
long_stops.head()

,current_stop,current_status,timestamp,stop_id,stop_name,cum_stop


In [15]:
def diff_calc(x):
    return x.iloc[-1] - x.iloc[0]

In [16]:
stop_time = trip_stops.groupby(["stop_id", "stop_name"]).agg(diff_calc)
stop_time

,,current_stop,current_status,timestamp,cum_stop
stop_id,stop_name,,,,
340001235OPP,Carpenter Close,0,0,0 days,0.0
340001237LKF,Minchery Road,0,0,0 days,0.0
340001244CNR,Catholic Church,0,0,0 days,0.0
340001252OUT,Church Cowley School,0,0,0 days,0.0
